### Workflow from genomes to families tables
* Setup dir structure
* Cluster ORFs and select reprseqs
* Create HMM profiles with uniclust
* Create db from set of profiles
* Run all vs all comparison of profiles
* Collect resulst of all-vs-all into table

This notebook does **not** contain running profiles on HHsuite-formated databases.

In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [2]:
### setup ###

### imports

import random
import sys
import os
pipeline_env_path = '/Users/kszczepaniak/Code/phage-pipeline-env'
lib_pp_path =       '/Users/kszczepaniak/Code/phage-pipeline-env/phage-pipeline'
sys.path.append(lib_pp_path)
import submodules.pdf.domain_split as ds
from lib_phage.clustering import cluster_proteins
from lib_phage.utils import setup_dir_tree, fetch_and_rename_protein_ids, build_hhr_table
from lib_phage.utils import process_phanotate_output, create_reprseq_profile_from_clustering
from lib_phage.utils import create_bash_script_to_parse_hhr_results, run_parsing_with_bash
from lib_phage.utils import concatenate_parsing_results, clean_clustering_partial_data
from lib_phage.logs import check_input_repr_prot_selection, validate_output_repr_prot_selection
from lib_phage.logs import check_input_all_vs_all_HMM, save_params_hhblits, validate_output_hhblits
from lib_phage.logs import validate_create_db, validate_search_all_vs_all, validate_input_ECF, validate_output_ECF
from lib_phage.prot_compare import save_individual_seqs, run_hhblits, build_hh_db, run_all_vs_all
from lib_phage.ecf_finder_wrapper import load_and_filter_data, store_scan_results

from lib_phage.repr_hits_pairwise import get_prob_cov

### run mode
run_mode = 'hhblits' # profile creation mode [mmseqs/hhblits]

### paths

# data sources

# cds_all_filepath = '/Users/kszczepaniak/Data/Phage/Rafal_Dropbox/data/input/combined/coding-seqs/cds-aa.fa.gz'
phanotate_filepath = '/Users/kszczepaniak/Data/Phage/phage-pipeline-input-dev/seq-test.phanotate.txt'

# work dirs
work_dir = '/Users/kszczepaniak/Data/Phage/phage-pipeline-workdir-dev/'
setup_dir_tree(work_dir)

# binaries and libraries

mmseqs_binpath  = 'mmseqs'
uniref_db_path  = '/Users/kszczepaniak/Data/Phage/db/UniRef30_2020_06/UniRef30_2020_06'
hhsuite_bins    = '/Users/kszczepaniak/Tools/hh-suite/build/bin'
hhsuite_scripts = '/Users/kszczepaniak/Tools/hh-suite/build/scripts'

/tmp/ directory already set up
/tmp/repr-proteins/ directory already set up
/tmp/mmseqs/ directory already set up
/tmp/all-by-all/ directory already set up
/tmp/all-by-all/individual-seqs/ directory already set up
/tmp/parse/ directory already set up
/tmp/prot-families/ directory already set up
/tmp/prot-families/pair_table_chunks/ directory already set up
/input/ directory already set up
/input/phanotate/ directory already set up
/input/coding-seqs/ directory already set up
/output/ directory already set up
/output/prot-families/ directory already set up
/output/prot-families/representative/ directory already set up
/output/prot-families/all-by-all/ directory already set up
//output/prot-families/functional/ directory already set up
//output/prot-families/families/ directory already set up
/output/prot-families/all-by-all/hhblits/ directory already set up
/output/prot-families/all-by-all/mmseqs/ directory already set up
/intermediate/ directory already set up
/intermediate/prot-famili

In [8]:
### Create protein db from Phanotate data ###

# translate Phanotate & compress with bgzip
process_phanotate_output(phanotate_filepath=phanotate_filepath, work_dir=work_dir)

All fasta translated. File compressed.


In [9]:
### Get representative proteins ###

# set clustering params
cluster_params_min_seqid   = 0.3
cluster_params_sensitivity = 7
cluster_params_coverage    = 0.95

# check input files integrity & if this step was already executed:
# if it was warn about data overwrite
if check_input_repr_prot_selection():

    # perform clustering
    clustering_filepath, clustering_msa_filepath = cluster_proteins(input_fasta_filepath=work_dir + 'input/coding-seqs/cds-aa.fa.gz',
                                           output_dirpath=work_dir + 'output/prot-families/representative',
                                           mmseqs_tempdir=work_dir + 'tmp/mmseqs',
                                           mmseqs_binpath=mmseqs_binpath,
                                           cluster_params_min_seqid=cluster_params_min_seqid,
                                           cluster_params_sensitivity=cluster_params_sensitivity,
                                           cluster_params_coverage=cluster_params_coverage,
                                           verbose=True)

    no_repr_prot, name_table_filepath = fetch_and_rename_protein_ids(work_dir, clustering_filepath, 
                                                                     work_dir + 'input/coding-seqs/cds-aa.fa.gz')

    # verify output and save log file
    validate_output_repr_prot_selection(work_dir=work_dir,
                                        output_dirpath=work_dir + 'output/prot-families/representative',
                                        cluster_params_min_seqid=cluster_params_min_seqid,
                                        cluster_params_sensitivity=cluster_params_sensitivity,
                                        cluster_params_coverage=cluster_params_coverage)

Creating db... Done!
Clustering... Done!
Generating a clustering table... Done!
Creating MSA... Done!
Validation success, log file stored.


In [28]:
### Create profiles from mmseqs clusters ###
# profile for singleton in clustering is the sequence itself

create_reprseq_profile_from_clustering(clustering_filepath, clustering_msa_filepath,
                                       cds_all_filepath = work_dir + 'input/coding-seqs/cds-aa.fa',
                                       profile_outdir = work_dir + 'intermediate/prot-families/profiles/mmseqs')

In [10]:
### Perform all vs all comparison ###
# create profiles for each protein with hhblits #

# set create profiles params
cpu  = 2 # max number of CPUs to be used in the step
n    = 1
mact = 0.35
p    = 90
qid  = 10
cov  = 10

if run_mode == 'hhblits': # execute this only when creating profiles with hhblits

    # validate previous step
    if check_input_all_vs_all_HMM(work_dir=work_dir, force=True):

        # execute current step
        save_individual_seqs(work_dir=work_dir)

        run_hhblits(work_dir=work_dir, hhsuite_bins=hhsuite_bins, hhsuite_scripts=hhsuite_scripts, cpu=cpu, 
                    uniref_db_path=uniref_db_path, n=n, mact=mact, p=p, qid=qid, cov=cov)

        # save params to log
        save_params_hhblits(work_dir=work_dir, n=n, mact=mact, p=p, qid=qid, cov=cov)

This will overwrite all data from this step. Proceed? [y/n]y
Clearing /Users/kszczepaniak/Data/Phage/phage-pipeline-workdir-dev/intermediate/prot-families/all-by-all...
Parameters saved, log file stored.


In [30]:
# Build db #

if run_mode == 'hhblits': # execute this only when creating profiles with hhblits

    # validate previous step
    if validate_output_hhblits(work_dir):

        # execute current step
        build_hh_db(work_dir=work_dir, hhsuite_bins=hhsuite_bins,
                    hhsuite_scripts=hhsuite_scripts, verbose=True, run_mode=run_mode)
        
elif run_mode == 'mmseqs':
    # execute current step
    build_hh_db(work_dir=work_dir, hhsuite_bins=hhsuite_bins,
                hhsuite_scripts=hhsuite_scripts, verbose=True, run_mode=run_mode)


Database already exists. Overwrite? [y/n]y
Database cleaned.
Concatenated a3m alignments.
Created HMM profiles.
Created column state (CS) sequence database.
DB sorted.
DB successfuly created.


In [31]:
# search all vs all #

# set all vs all search params
cpu  = 2 # max number of CPUs to be used in the step
n    = 1
p    = 90

if validate_create_db(work_dir=work_dir):

    run_all_vs_all(work_dir=work_dir, hhsuite_bins=hhsuite_bins, 
                   hhsuite_scripts=hhsuite_scripts, cpu=cpu, n=n, 
                   p=p, a3m_wildcard='reprseq*a3m', run_mode=run_mode)

Parameters saved, log file stored.


In [32]:
# create results table #

# check if previous step complete
if validate_search_all_vs_all(work_dir, run_mode=run_mode):
    
    # create results table
    build_hhr_table(work_dir, run_mode=run_mode)


HHblits all-vs-all step complete. Updated status.


In [3]:
# repr-hits-pairwise #
import pandas as pd
import numpy as np

## define paths
output_dir = work_dir + 'output/'
inter_dir = work_dir + 'intermediate/'
all_by_all_output_dir = output_dir + 'prot-families/all-by-all/' + run_mode + '/'
families_output_dir = output_dir + 'prot-families/families/'

# load hhr_table
table_hhr_filename = all_by_all_output_dir + 'table-hhr.txt'
table_hhr = pd.read_csv(table_hhr_filename, sep=',')
table_hhr

,qname,qstart,qend,qlength,sname,sstart,send,slength,pident,bitscore,eval,prob,pval
0,reprseq00001,1,86,86,reprseq00001,1,86,86,100,246.5,5.300000e-47,100.0,7.700000e-51
1,reprseq00001,30,44,86,reprseq14105,27,41,73,20,25.8,9.300000e-01,56.2,1.100000e-04
2,reprseq00002,1,125,125,reprseq00002,1,125,125,100,374.6,8.800000e-65,100.0,1.300000e-68
3,reprseq00002,1,125,125,reprseq13080,1,125,138,40,364.7,5.900000e-62,100.0,8.200000e-66
4,reprseq00002,1,125,125,reprseq17146,28,158,158,33,367.2,3.400000e-61,100.0,4.600000e-65
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382435,reprseq34463,55,76,200,reprseq32916,39,60,113,27,29.7,1.100000e+00,51.5,1.500000e-04
1382436,reprseq34464,1,187,187,reprseq34464,1,187,187,100,463.6,1.400000e-72,100.0,2.100000e-76
1382437,reprseq34464,9,183,187,reprseq06995,1,181,181,32,432.2,8.900000e-68,100.0,1.400000e-71
1382438,reprseq34464,6,73,187,reprseq13037,48,109,201,27,34.7,5.900000e-01,63.0,7.000000e-05


In [10]:
# DEV
table_hhr.groupby(['qname', 'sname']).size().idxmax()


('reprseq00027', 'reprseq01409')

In [13]:
# DEV
df1 = table_hhr[(table_hhr['qname'] == 'reprseq00027') & (table_hhr['sname'] == 'reprseq01409')]
df1

,qname,qstart,qend,qlength,sname,sstart,send,slength,pident,bitscore,eval,prob,pval
489,reprseq00027,12,35,82,reprseq01409,293,316,323,13,37.0,0.010,89.9,0.000002
492,reprseq00027,11,39,82,reprseq01409,240,268,323,24,36.9,0.011,89.8,0.000002
520,reprseq00027,13,37,82,reprseq01409,194,218,323,20,35.5,0.020,87.7,0.000003
668,reprseq00027,13,36,82,reprseq01409,132,155,323,17,29.0,0.390,64.2,0.000065


In [16]:
# DEV
df1 = df1.assign(qcov = lambda x: (x.qend - x.qstart + 1) / x.qlength)
df1

,qname,qstart,qend,qlength,sname,sstart,send,slength,pident,bitscore,eval,prob,pval,qcov
489,reprseq00027,12,35,82,reprseq01409,293,316,323,13,37.0,0.010,89.9,0.000002,0.292683
492,reprseq00027,11,39,82,reprseq01409,240,268,323,24,36.9,0.011,89.8,0.000002,0.353659
520,reprseq00027,13,37,82,reprseq01409,194,218,323,20,35.5,0.020,87.7,0.000003,0.304878
668,reprseq00027,13,36,82,reprseq01409,132,155,323,17,29.0,0.390,64.2,0.000065,0.292683


In [17]:
# DEV
df1[df1['qcov'] == df1['qcov'].max()]

,qname,qstart,qend,qlength,sname,sstart,send,slength,pident,bitscore,eval,prob,pval,qcov
492,reprseq00027,11,39,82,reprseq01409,240,268,323,24,36.9,0.011,89.8,0.000002,0.353659


In [32]:
## get unique pairs
## get from table_hhr qname-sname unique pairs of ids

pair_table = table_hhr[['qname', 'sname']]
pair_table = pair_table[pair_table['qname'] != pair_table['sname']]
pair_table = pair_table.drop_duplicates()
pair_table

,qname,sname
1,reprseq00001,reprseq14105
3,reprseq00002,reprseq13080
4,reprseq00002,reprseq17146
6,reprseq00003,reprseq27929
8,reprseq00004,reprseq08141
...,...,...
1382434,reprseq34463,reprseq04091
1382435,reprseq34463,reprseq32916
1382437,reprseq34464,reprseq06995
1382438,reprseq34464,reprseq13037


In [33]:
#test#
pair_table_test = pair_table[:100]

In [34]:
# create script to run in parallel
n_cores = 4
n_subsets = 5

In [35]:
#!! pair_table_test !!#

# divide pair table into sub-tables
chunk_size = int(len(pair_table_test) / n_cores)
pair_table_parts = []
for i in range(n_cores-1):
    pair_table_parts.append(pair_table_test[i*chunk_size:(i+1)*chunk_size])
pair_table_parts.append(pair_table_test[(n_cores-1)*chunk_size:])

parts_dir = work_dir + 'tmp/prot-families/pair_table_chunks/'
for i, pair_table_part in enumerate(pair_table_parts):
    pair_table_part.to_csv(parts_dir + 'pair-table-' + str(i) + '.csv', index=False)

In [38]:
# now divide into sub-subtables for smaller mem usage
for i in range(n_cores):
    pair_table_part = pd.read_csv(parts_dir + 'pair-table-' + str(i) + '.csv')
    pair_table_subsets = np.array_split(pair_table_part, n_subsets)

    for j, pair_sub in enumerate(pair_table_subsets):
        pair_sub.to_csv(parts_dir + 'pair-table-' + str(i) + '-' + str(j) + '.csv', index=False)
    os.remove(parts_dir + 'pair-table-' + str(i) + '.csv')

In [39]:
# create subtables of results to optimize mem usages
for i in range(n_cores):
    for j in range(n_subsets):
        pair_sub = pd.read_csv(parts_dir + 'pair-table-' + str(i) + '-' + str(j) + '.csv')
        qname_list = list(pair_sub['qname'].unique())
        sname_list = list(pair_sub['sname'].unique())
        
        df1 = table_hhr[(table_hhr['qname'].isin(qname_list)) & (table_hhr['sname'].isin(sname_list))]
        df2 = table_hhr[(table_hhr['qname'].isin(sname_list)) & (table_hhr['sname'].isin(qname_list))]
        df = pd.concat([df1, df2])
        df.drop_duplicates(inplace=True)
        
        df.to_csv(parts_dir + 'table-hhr-' + str(i) + '-' + str(j) + '.csv', index=False)

In [40]:
# run in parallel python script to process sub-table
script_filepath =  work_dir + 'tmp/prot-families/run-pairwise-hits.sh'
prob_threshold = 50

cmd = '#!/bin/bash\n\n'
cmd += 'source ' + pipeline_env_path + '/bin/activate\n'

for i, pair_table_part in enumerate(pair_table_parts):
    pair_table_path = parts_dir + 'pair-table-' + str(i) + '.csv'    
    cmd += 'nohup python3 {}/phage-pipeline/lib_phage/run_hits_pairwise_single_table.py {} {} {} {} {} {} &\n'.format(
    pipeline_env_path, work_dir, run_mode, i, prob_threshold, n_subsets, lib_pp_path)

with open(script_filepath, 'w') as file_sh:
    file_sh.write(cmd)

In [44]:
table_hhr[(table_hhr['sname'] == 'reprseq00001') & (table_hhr['qname'] == 'reprseq14105')]

,qname,qstart,qend,qlength,sname,sstart,send,slength,pident,bitscore,eval,prob,pval
538092,reprseq14105,27,42,73,reprseq00001,30,45,86,19,27.7,0.22,71.8,0.000035


In [14]:
pair_table_test = pd.read_csv(parts_dir + 'pair-table-' + str(1) + '.csv', sep=',')
list_df = np.array_split(pair_table_test, 5)
list_df

[          qname         sname
 0  reprseq00010  reprseq10204
 1  reprseq00011  reprseq17136
 2  reprseq00011  reprseq09566
 3  reprseq00011  reprseq22390
 4  reprseq00011  reprseq22242,
           qname         sname
 5  reprseq00011  reprseq19820
 6  reprseq00011  reprseq10972
 7  reprseq00011  reprseq15008
 8  reprseq00011  reprseq05448
 9  reprseq00011  reprseq10207,
            qname         sname
 10  reprseq00011  reprseq30158
 11  reprseq00011  reprseq15003
 12  reprseq00011  reprseq22993
 13  reprseq00012  reprseq19899
 14  reprseq00015  reprseq30579,
            qname         sname
 15  reprseq00016  reprseq09572
 16  reprseq00016  reprseq11171
 17  reprseq00017  reprseq00366
 18  reprseq00017  reprseq00464
 19  reprseq00018  reprseq00296,
            qname         sname
 20  reprseq00018  reprseq13097
 21  reprseq00018  reprseq09129
 22  reprseq00018  reprseq29110
 23  reprseq00018  reprseq01388
 24  reprseq00018  reprseq06150]

In [30]:
650000 / 40

16250.0

In [20]:
### DEBUG

pair_table_test = pd.read_csv(parts_dir + 'pair-table-' + str(1) + '.csv', sep=',')

# divide into subtables to process & save idependently
pair_table_subsets = np.array_split(pair_table_test, 5)

for j, pair_sub in enumerate(pair_table_subsets):
    
    pair_results = {}
    for i, pair in pair_sub.iterrows():

        this_pair_qname = pair['qname']
        this_pair_sname = pair['sname']

        prob_ab=0
        prob_ba=0
        scov_ab=0
        scov_ba=0
        qcov_ab=0
        qcov_ba=0
        pident_ab=0
        pident_ba=0

        this_pair_hits_ab = table_hhr[(table_hhr['qname'] == this_pair_qname) & (table_hhr['sname'] == this_pair_sname)]

        this_pair_prob_cov_ab = get_prob_cov(this_pair_hits_ab, prob_threshold = prob_threshold)
        if this_pair_prob_cov_ab:
            prob_ab = this_pair_prob_cov_ab[0]
            scov_ab = this_pair_prob_cov_ab[1]
            qcov_ab = this_pair_prob_cov_ab[2]
            pident_ab = this_pair_prob_cov_ab[3]


        this_pair_hits_ba = table_hhr[(table_hhr['sname'] == this_pair_qname) & (table_hhr['qname'] == this_pair_sname)]

        this_pair_prob_cov_ba = get_prob_cov(this_pair_hits_ba, prob_threshold = prob_threshold)
        if this_pair_prob_cov_ba:
            prob_ba = this_pair_prob_cov_ba[0]
            scov_ba = this_pair_prob_cov_ba[1]
            qcov_ba = this_pair_prob_cov_ba[2]
            pident_ba = this_pair_prob_cov_ba[3]


        min_prob = 0
        if (prob_ab and prob_ba):
            min_prob = min(prob_ab, prob_ba)
        this_pair_prob = min_prob
        this_pair_scov = min(scov_ab, scov_ba)
        this_pair_qcov = min(qcov_ab, qcov_ba)
        min_pident = 0
        if (pident_ab and pident_ba):
            min_pident = min(pident_ab, pident_ba)
        this_pair_pident = min_pident

        pair_results[i] = [this_pair_qname, this_pair_sname, this_pair_qcov,
                           this_pair_scov, this_pair_prob, this_pair_pident]

    ### now to dataframe and drop to file
    pair_results_df = pd.DataFrame.from_dict(pair_results, orient='index',
                                             columns=['qname', 'sname', 'qcov', 'scov', 'prob', 'pident'])
    display(pair_results_df)
    # pair_results_df.to_csv(families_output_dir + output_filename, index=False, float_format="%.4f")


,qname,sname,qcov,scov,prob,pident
0,reprseq00010,reprseq10204,0.157895,0.162679,0.782,0.38
1,reprseq00011,reprseq17136,0.952381,0.923077,0.999,0.34
2,reprseq00011,reprseq09566,0.953846,0.953846,0.999,0.24
3,reprseq00011,reprseq22390,0.830769,0.846154,0.999,0.13
4,reprseq00011,reprseq22242,0.261538,0.261538,0.678,0.35


,qname,sname,qcov,scov,prob,pident
5,reprseq00011,reprseq19820,0.276923,0.305085,0.642,0.32
6,reprseq00011,reprseq10972,0.000000,0.000000,0.000,0.00
7,reprseq00011,reprseq15008,0.262295,0.246154,0.606,0.25
8,reprseq00011,reprseq05448,0.292308,0.323077,0.502,0.24
9,reprseq00011,reprseq10207,0.000000,0.000000,0.000,0.00


,qname,sname,qcov,scov,prob,pident
10,reprseq00011,reprseq30158,0.156522,0.156522,0.515,0.22
11,reprseq00011,reprseq15003,0.000000,0.000000,0.000,0.00
12,reprseq00011,reprseq22993,0.000000,0.000000,0.000,0.00
13,reprseq00012,reprseq19899,0.830508,0.830508,0.998,0.31
14,reprseq00015,reprseq30579,0.488095,0.511905,0.758,0.31


,qname,sname,qcov,scov,prob,pident
15,reprseq00016,reprseq09572,0.604478,0.595588,0.995,0.23
16,reprseq00016,reprseq11171,0.611940,0.672131,0.961,0.26
17,reprseq00017,reprseq00366,0.899441,0.905028,1.000,0.41
18,reprseq00017,reprseq00464,0.835821,0.766169,1.000,0.31
19,reprseq00018,reprseq00296,0.510040,0.534137,1.000,0.28


,qname,sname,qcov,scov,prob,pident
20,reprseq00018,reprseq13097,0.570281,0.542169,1.000000,0.230000
21,reprseq00018,reprseq09129,0.819277,0.823293,0.993804,0.299163
22,reprseq00018,reprseq29110,0.518072,0.534137,1.000000,0.240000
23,reprseq00018,reprseq01388,0.477912,0.473896,0.999000,0.280000
24,reprseq00018,reprseq06150,0.337349,0.465863,0.956000,0.190000


In [12]:
pair_results_df[pair_results_df['qname'] == 'reprseq00018']

,qname,sname,qcov,scov,prob,pident
19,reprseq00018,reprseq00296,0.510040,0.534137,1.000000,0.280000
20,reprseq00018,reprseq13097,0.570281,0.542169,1.000000,0.230000
21,reprseq00018,reprseq09129,0.819277,0.823293,0.993804,0.299163
22,reprseq00018,reprseq29110,0.518072,0.534137,1.000000,0.240000
23,reprseq00018,reprseq01388,0.477912,0.473896,0.999000,0.280000
24,reprseq00018,reprseq06150,0.337349,0.465863,0.956000,0.190000


In [97]:
# there are some paths that are used further in pipeline #

import pandas as pd
import numpy as np

## define paths
output_dir = work_dir + 'output/'
prot_families_dir = output_dir + 'prot-families/'
prot_families_temp_dir = work_dir + 'tmp/prot-families/'

inter_dir = work_dir + 'intermediate/'
all_by_all_output_dir = output_dir + 'prot-families/all-by-all/' + run_mode + '/'
families_output_dir = output_dir + 'prot-families/families/'

## load representative mappings
repr_dir = work_dir + 'output/prot-families/representative/'
repr_table_filename = repr_dir + 'name-table.txt'
repr_table = pd.read_csv(repr_table_filename, sep=',')
nr = len(repr_table)
nrd = len(str(nr))

table_hhr_filename = all_by_all_output_dir + 'table-hhr.txt'
table_hhr = pd.read_csv(table_hhr_filename, sep=',')
table_hhr

,qname,qstart,qend,qlength,sname,sstart,send,slength,pident,bitscore,eval,prob,pval
0,reprseq00001,1,86,86,reprseq00001,1,86,86,100,246.5,5.300000e-47,100.0,7.700000e-51
1,reprseq00001,30,44,86,reprseq14105,27,41,73,20,25.8,9.300000e-01,56.2,1.100000e-04
2,reprseq00002,1,125,125,reprseq00002,1,125,125,100,374.6,8.800000e-65,100.0,1.300000e-68
3,reprseq00002,1,125,125,reprseq13080,1,125,138,40,364.7,5.900000e-62,100.0,8.200000e-66
4,reprseq00002,1,125,125,reprseq17146,28,158,158,33,367.2,3.400000e-61,100.0,4.600000e-65
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382435,reprseq34463,55,76,200,reprseq32916,39,60,113,27,29.7,1.100000e+00,51.5,1.500000e-04
1382436,reprseq34464,1,187,187,reprseq34464,1,187,187,100,463.6,1.400000e-72,100.0,2.100000e-76
1382437,reprseq34464,9,183,187,reprseq06995,1,181,181,32,432.2,8.900000e-68,100.0,1.400000e-71
1382438,reprseq34464,6,73,187,reprseq13037,48,109,201,27,34.7,5.900000e-01,63.0,7.000000e-05
